In [ ]:
import sys
from collections import OrderedDict
from pyexcel_ods import get_data
from config_rm import *
from redminelib import Redmine

In [ ]:
redmine = Redmine(rm_server_url,key=rm_key_txt)
projects = redmine.project.all()

In [ ]:
print("Proyectos:")
for p in projects:
    print ("\t",p.identifier," \t| ",p.name)

my_project = redmine.project.get(rm_project_id_str)
print ("Obtenemos proyecto: ",my_project.identifier," | ",my_project.name)

In [ ]:
import datetime
def date_by_adding_business_days(from_date, add_days):
    business_days_to_add = add_days
    current_date = from_date
    while business_days_to_add > 0:
        current_date += datetime.timedelta(days=1)
        weekday = current_date.weekday()
        if weekday >= 5: # sunday = 6
            continue
        business_days_to_add -= 1
    return current_date


my_project = redmine.project.get(rm_project_id_str)
print ("Obtenemos proyecto: ",my_project.identifier," | ",my_project.name)
done_issues = []
issues_list = []
issues_number = len(my_project.issues)
for tsk in my_project.issues:
    thisTskRelations = tsk.relations
    thisTskInputRelations = list(filter(lambda x: x.issue_to_id == tsk.id and x.relation_type=='precedes', thisTskRelations))
    relations = []
    if len(thisTskInputRelations)>0:
        print("bypass "+tsk.subject)
        for rel in thisTskInputRelations:
            relations += [rel.issue_id]
    
    else:
        print("process "+tsk.subject)
        done_issues += [tsk.id]
        if not(hasattr(tsk, 'start_date')):
            tsk.start_date = datetime.date.today()

        print("calculate due date from "+str(tsk.start_date)+" + "+str(tsk.estimated_hours))
        tmp = date_by_adding_business_days(tsk.start_date, tsk.estimated_hours/8)
        print("result = "+str(tmp))
        tsk.due_date = tmp
        tsk.save()

    issues_list += [{'tsk': tsk, 'relations': relations}]
        
print ("------------------ RECURSIVE PROCESS -------------------")
while len(done_issues) < issues_number:
    print("--- iteration ---")
    for tsktuple in issues_list:
        tskid = tsktuple['tsk'].id
        if (tskid not in done_issues):
            relations = tsktuple['relations']
            for r in relations:
                if r in done_issues:
                    tsk = redmine.issue.get(tskid)
                    print("process "+tsk.subject)                    
                    print("calculate due date from "+str(tsk.start_date)+" + "+str(tsk.estimated_hours))
                    tmp = date_by_adding_business_days(tsk.start_date, tsk.estimated_hours/8)
                    print("result = "+str(tmp))
                    tsk.due_date = tmp
                    tsk.save()
                    done_issues += [tskid]
                

print("Done")
        
    